<a href="https://colab.research.google.com/github/Jinzhao-Yu/BioStat615/blob/main/BIOSTAT615_Lecture_4_Fall_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BIOSTAT615 Lecture 4 - R

## 1. Matrix multiplication examples

In [ ]:
#' naive.matrix.mult.v1() : multiply two matrices
#' @param A, B : two matrices with ncol(A)==nrow(B) 
#' @return Product of A and B 
naive.matrix.mult.v1 = function(A, B) {
  n = nrow(A); p = ncol(A); m = ncol(B)
  stopifnot(p == nrow(B)) # check the dimension
  C = matrix(0, n, m) 
  for(i in 1:n) { # triple-loop implementation
    for(j in 1:m) {
      for(k in 1:p) {
        C[i,j] = C[i,j] + A[i,k] * B[k,j]
      }
    }
  }
  return(C)
}

In [ ]:
# check for correctness
A = matrix(1:9,3,3)
B = matrix(1:9,3,3)
print(naive.matrix.mult.v1(A, B))
print(A %*% B) # R built-in implementation

Is the matrix multiplication algorithm working correctly?

In [ ]:
# check for efficiency at a small scale
A = matrix(rnorm(10000),100,100)
B = matrix(rnorm(10000),100,100)

## evaluate the computational time for naive multiplication
system.time(naive.matrix.mult.v1(A, B))

## compare with stanard matrix multiplication
system.time(A %*% B)

In [ ]:
# 3 times larger in each dimension
A = matrix(rnorm(90000),300,300)
B = matrix(rnorm(90000),300,300)

## evaluate the computational time for naive multiplication
system.time(naive.matrix.mult.v1(A, B))

## compare with stanard matrix multiplication
system.time(A %*% B)

Is the matrix multiplication algorithm working efficiently?

The above code is slow probably because there are multiple layers of loop.
Instead of using triple loops, let's change it to double loop by leveraging the built-in function `sum()`

In [ ]:
#' naive.matrix.mult.v2() : multiply two matrices
#' @param A, B : two matrices with ncol(A)==nrow(B) 
#' @return Product of A and B 
naive.matrix.mult.v2 = function(A, B) {
  n = nrow(A); p = ncol(A); m = ncol(B)
  stopifnot(p == nrow(B))
  C = matrix(0, n, m)
  for(i in 1:n) {
    for(j in 1:m) {
      C[i,j] = sum(A[i,] * B[,j]) # the only difference
    }
  }
  return(C)
}

In [ ]:
A = matrix(rnorm(10000),100,100)
B = matrix(rnorm(10000),100,100)

## evaluate the computational time for triple-loop multiplication
system.time(naive.matrix.mult.v1(A, B))

## evaluate the computational time for double-loop and sum
system.time(naive.matrix.mult.v2(A, B))

## compare with stanard matrix multiplication
system.time(A %*% B)

In [ ]:
A = matrix(rnorm(90000),300,300)
B = matrix(rnorm(90000),300,300)

## evaluate the computational time for triple-loop multiplication
system.time(naive.matrix.mult.v1(A, B))

## evaluate the computational time for double-loop and sum
system.time(naive.matrix.mult.v2(A, B))

## compare with stanard matrix multiplication
system.time(A %*% B)

Are there differences between the two versions of matrix multiplication algorithms?

How do they differ from built-in matrix multiplication in R?

In [ ]:
A = matrix(rnorm(1000000),1000,1000)
B = matrix(rnorm(1000000),1000,1000)

#system.time(naive.matrix.mult.v1(A, B)) ## too long

## evaluate the computational time for double-loop and sum
system.time(naive.matrix.mult.v2(A, B))

## compare with stanard matrix multiplication
system.time(A %*% B)

Do you see noticable difference in time complexity?

Why do you think we see such a large gap in efficiency?

## 2. Checking BLAS/LAPACK version

You may check the version of matrix libraries `BLAS` and `LAPACK` using the following command:

Typically, OpenBLAS and Intel MKL are much faster than GNU BLAS.

In [ ]:
sessionInfo()

## 3. Linear Regression

In [ ]:
## linear regression with 100 observations
y = rnorm(100)
x = rnorm(100)
summary(lm(y~x))

In [ ]:
## linear regression with 5M observations
y = rnorm(5000000)
x = rnorm(5000000)

## evaluate the time
system.time(print(summary(lm(y~x))))

## 4. A faster implementation for simple linear regression


In [ ]:
#' fastSimpleLinearRegression()
#' @param y : A vector of response variable
#' @param x : A vector predictor variable, same length with y
#' @return A list containing OLS estimates for y = \mu + x + e
fastSimpleLinearRegression <- function(y, x) {
  y = y - mean(y)
  x = x - mean(x)
  n = length(y)
  stopifnot(length(x) == n)       # for error handling
  s2y = sum( y * y ) / ( n - 1 )  # \sigma_y^2
  s2x = sum( x * x ) / ( n - 1 )  # \sigma_x^2
  sxy = sum( x * y ) / ( n - 1 )  # \sigma_xy
  rxy = sxy / sqrt( s2y * s2x )   # \rho_xy
  b = rxy * sqrt( s2y / s2x )
  se.b = sqrt( s2y * ( 1 - rxy * rxy ) / (n-2) / s2x )
  tstat = rxy * sqrt( ( n - 2 ) / ( 1 - rxy * rxy ) )
  p = pt( abs(tstat) , n - 2 , lower.tail=FALSE )*2
  return(list( beta = b , se.beta = se.b , t.stat = tstat, p.value = p ))
}

In [ ]:
## evaluation with the faster version
system.time(print(fastSimpleLinearRegression(x,y)))

## 5. Solving linear system with matrix decomposition


### Linear regression with SVD 

$$
X  =  UDV^T    \\
\hat{\boldsymbol{\beta}} = (X^TX)^{-1}X^T\mathbf{y} 
$$

In [ ]:
#' svd.lm.fit () - Solve linear system with SVD
#' @param x - design matrix
#' @param y - response variable 
#' @return estimated \beta for y ~ X\beta + e
svd.lm.fit <- function(x,y){
  res_svd <- svd(x)
  tuy <- crossprod(res_svd$u,y) # tuy = t(U) %*% y
  return(res_svd$v%*%(tuy/res_svd$d)) # V %*% D^{-1} %*% t(U) %*% y
}

## Linear regression with QR decomposition

$$    
X = QR \\
R\hat{\boldsymbol{\beta}} = Q^T\mathbf{y}  
$$

where $R$ is upper-trigular matrix (i.e. quadratic time complexity to solve)

In [ ]:
#' qr.lm.fit () - Solve linear system with QR decomposition
#' @param x - design matrix
#' @param y - response variable 
#' @return estimated \beta for y ~ X\beta + e
qr.lm.fit <- function(x,y){
  res_qr <- qr(x)  
  return(qr.coef(res_qr,y)) ## solves R\beta = Qy in its specific way
}

### Linear regression with Cholesky decomposition

Define
$$
    A = X^T X  = U^T U \\
    \mathbf{b} = X^{T} \mathbf{y}\\
$$

Solve
$$
    U^T \mathbf{z} = \mathbf{b}\\
    U\hat{\boldsymbol{\beta}} = \mathbf{z}
$$

Note that both equations are solving against (lower or upper) triangular matrices, which can be computed efficiently.

In [ ]:
#' chol.lm.fit () - Solve linear system with Cholesky decomposition
#' @param x - design matrix
#' @param y - response variable 
#' @return estimated \beta for y ~ X\beta + e
chol.lm.fit <- function(x,y){
  tXX <- crossprod(x)     # t(x) %*% x
  tXY <- crossprod(x,y)   # t(x) %*% y
  U <- chol(tXX)          
  z <- forwardsolve(U,tXY,upper.tri=TRUE,transpose=TRUE) # solve t(U) %*% z = tXY for z
  return(backsolve(U,z))  # solve U %*% beta = z for beta
}

See [forwardsolve/backsolve documentation](https://www.rdocumentation.org/packages/base/versions/3.6.2/topics/backsolve) to understand how to use these functions.

### Evaluation of different methods

In [ ]:
## number of observations
n = 10000L

## number of variables
p = 800L

## number of true predictors
q = 5L

## correlation between the variables
rho = 0.99

We want to simulate an example with
$$
X \sim \mathcal{N}(0, V)
$$
where 
$$
V_{ij} = \left\{
\begin{array}{ll}
  \rho & i \ne j \\
  1 & i = j 
\end{array}
\right.
$$

and

$$Y = X\beta + \epsilon$$

In [ ]:
# set up a linear system with strong correlation
# X is n*p matrix with rho as covariance between columns
X = matrix(rnorm(p*n,sd=sqrt(1-rho)),nrow=n,ncol=p) + matrix(rnorm(n,sd=sqrt(rho)),nrow=n,ncol=p)
beta = c(rep(c(1,-1),length=q),rep(0,length=p-q)) # beta : first q nonzero, rest is zero
eps = rnorm(n,sd=1) # eps : standard normal random noise
Y = X%*%beta + eps  # Y ~ Xb + e

In [ ]:
## evaluate the time - may take long
tm_default <- system.time(fit_default_lm <- lm.fit(X,Y))
tm_svd_lm <- system.time(fit_svd_lm <- svd.lm.fit(X,Y))
tm_qr_lm <- system.time(fit_qr_lm <- qr.lm.fit(X,Y))
tm_chol_lm <- system.time(fit_chol_lm <- chol.lm.fit(X,Y))

In [ ]:
## check whether the estimated coefficients make sense
print( rbind(fit_default_lm$coefficients[1:10],
             fit_svd_lm[1:10],
             fit_qr_lm[1:10],
             fit_chol_lm[1:10]
             ))

In [ ]:
## print the evaluation results in terms of MSE and elapsed time
tab = data.frame(mse = c(lm = mean((fit_default_lm$coef-beta)^2),
                         svd = mean((fit_svd_lm-beta)^2),
                         qr = mean((fit_qr_lm - beta)^2),
                         chol = mean((fit_chol_lm-beta)^2)),
                 elapsed = c(lm = tm_default[3],
                             svd = tm_svd_lm[3],
                             qr = tm_qr_lm[3],
                             chol = tm_chol_lm[3])
)
print(tab)